In [23]:
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
from dotenv import load_dotenv

load_dotenv()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = ChatOpenAI(model="doubao-seed-1-6-flash-250828")

vectorstore_loaded = Milvus(
    embeddings,
    connection_args={"host": "localhost", "port": "19530"},
    collection_name="huozhe_collection",
)

retriever = vectorstore_loaded.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 10, "score_threshold": 0.3}  # 检索最相似的10个文档，默认L2距离，越小越相似
)

In [24]:
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="huozhe_tool",
    description="一个关于作家余华的小说《活着》，其中包含了这本小说的全部内容。",
)

tools = [retriever_tool]

In [25]:
from typing import TypedDict, Annotated, Sequence, Literal
from langchain_core.messages import BaseMessage
from langgraph.graph import add_messages
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [26]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

def agent(state):
    messages = state["messages"]
    model = llm.bind_tools(tools)
    response = model.invoke(messages)
    return {"messages": [response]}


def should_continue(state):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

tool_node = ToolNode(tools)
graph = StateGraph(AgentState)

graph.add_node("agent", agent)
graph.add_node("tools", tool_node)

graph.set_entry_point("agent")
graph.add_conditional_edges("agent", should_continue, {
    "tools": "tools",
    END: END
})
graph.add_edge("tools", "agent")

app = graph.compile()

In [27]:
result = app.invoke({
    "messages": [
        HumanMessage(content="请告诉我活着中有庆是怎么死的？")
]})["messages"][-1].content

print(result)

No index params provided. Could not determine relevance function. Use L2 distance as default.


在《活着》中，有庆是因为给县长夫人献血而被抽血过多致死的。具体情节如下：

有庆的学校校长（县长的女人）在医院生孩子时大出血，学校组织五年级学生去医院献血。有庆为了表现积极，第一个跑到医院，却因“不遵守纪律”被老师训斥。但他仍焦急等待，当医生发现之前的人血型都不合适时，有庆主动上前，最终被连续抽血。由于他年纪小、身体尚未发育完全，加上失血过多，在献完血后不久便倒下，最终抢救无效死亡。

福贵后来得知消息，抱着儿子冰冷的身体悲痛欲绝。家珍因儿子的死再次病重，这也成为福贵人生中又一个沉重的打击。余华通过这一情节，深刻展现了特殊年代背景下个体命运的脆弱与无奈。
